# Week 7. Problem set

**Evgeny Bobkunov SD-03 e.bobkunov@innopolis.university**


Consider the following declarations:

```haskell
type Name = String
data Grade = A | B | C | D
data Student = Student Name Grade
data Point2D = Point2D Int Int

data Status a
  = Stop
  | Warn [String] a
  
dup f x = f x x
dip f x = f x (f x x)
twice f x = f (f x)
```

1. Specify the (most generic) types of `dup`, `dip`, and `twice`.

`dup`

- `dup` takes a function `f` and a value `x`.

- `f` is applied to `x` twice, so the function `f` must take two arguments of the same type as `x`.

Thus, the type of `dup` is:

```haskell
dup :: (a -> a -> b) -> a -> b
```

`dip`

- `dip` takes a function `f` and a value `x`.

- The function `f` is applied to `x` twice inside itself, so `f` must take two arguments of the same type, and it must also return the same type as the input for the second argument of `f` to be valid.

Thus, the type of `dip` is:

```haskell
dip :: (a -> a -> a) -> a -> a
```

`twice`

- `twice` takes a function `f` and a value `x`.

- `f` is applied to `x`, and then `f` is applied again to the result of the first application.

Thus, the type of `dip` is:

```haskell
twice :: (a -> a) -> a -> a
```

In [4]:
type Name = String
data Grade = A | B | C | D
data Student = Student Name Grade
data Point2D = Point2D Int Int

data Status a
  = Stop
  | Warn [String] a
  
dup f x = f x x
dip f x = f x (f x x)
twice f x = f (f x)

dup :: (a -> a -> b) -> a -> b
dip :: (a -> a -> a) -> a -> a
twice :: (a -> a) -> a -> a


2. Infer the type for each of the following expressions or specify a type error. Justify your answer by providing a step-by-step reasoning process. Assume that the type of integer literals is `Int`:

(a) `dup Point2D`

`dup` expects a function `f` of type `(a -> a -> b)` (a function that takes two arguments of type `a` and returns a value of type `b`) and then applies `f` to two values of type `a`

The constructor `Point2D` has the following type:

```haskell
Point2D :: Int -> Int -> Point2D
```

This means `Point2D` is a function that takes two `Int` values and returns a value of type `Point2D`.

Since `Point2D` is a function of type `Int -> Int -> Point2D`, this fits perfectly: `dup` will apply `Point2D` to two values of type `Int`.

Since `dup Point2D` will create a function that takes an `Int` and applies `Point2D` to two `Int`s, the final type of the expression is:

```haskell
dup Point2D :: Int -> Point2D
```

(b) `dup (dup (+)) 3`

The `(+)` function has the following type:

```haskell
(+) :: Num a => a -> a -> a
```

This means that `(+)` takes two numbers of type `a` (where `a` must be an instance of the `Num` typeclass) and returns a value of type `a`.

Now, apply `dup` to `(+)`:

```haskell
dup (+)
```

- The function `dup` expects a function of type `(a -> a -> b)`. 
- The function `(+)` fits this type with `a = Num a`, and `b = a` (since `(+)` returns a number of the same type it takes as input).

Therefore, `dup (+)` will return a function that takes one argument of type `a` and applies `(+)` to that argument twice.

Thus, the type of `dup (+)` is:

```haskell
dup (+) :: Num a => a -> a
```

Next, let's apply `dup` again to `dup (+)`:

```haskell
dup (dup (+))
```

- The type of `dup` is `(a -> a -> b) -> a -> b`. Here, `dup (+)` has the type `Num a => a -> a`.

- This is not compatible with what `dup` expects, because `dup` needs a function that takes **two arguments**, but `dup (+)` is a function that takes only **one argument**.

Therefore, the expression `dup (dup (+)) 3` results in a *type error* because `dup (+)` is a unary function, and `dup` expects a binary function as its first argument.

(c) `twice dip`

To apply `twice dip`, `twice` expects `dip` to have the type `(a -> a)`, meaning it must be a function that takes **one argument** and returns **one argument**. However, the type of `dip` is:

```haskell
dip :: (a -> a -> a) -> a -> a
```

Therefore, the expression `twice dip` results in a *type error* due to the type mismatch between `twice`'s expected argument and `dip`'s actual type.

(d) `dip dip`

Clearly, `dip` itself is **not** a binary function. It takes a binary function as its argument but is **not** a binary function by itself. Therefore, trying to pass `dip` as the first argument to `dip` results in a *type mismatch*.


(e) `twice twice twice`

`twice` takes a function of type `(a -> a)` and returns a new function of the same type: `(a -> a)`.

Let's apply `twice` to `twice`:

- The outer `twice` has type `twice :: (a -> a) -> a -> a`.
- We apply it to `twice`, which has the type `(a -> a)`. So the result of `twice twice` will be a function.

To evaluate this, let's instantiate `a` in the type of `twice`:

```haskell
twice :: ((a -> a) -> (a -> a)) -> (a -> a)
```

After applying `twice` to `twice`, we get:

```haskell
twice twice :: (a -> a) -> (a -> a)
```

So the result of `twice twice` is a function that takes a function `(a -> a)` and returns another function `(a -> a)`.

Next, we apply the result of `twice twice` to another `twice`:

```haskell
(twice twice) twice
```

Applying `(twice twice)` to `twice` gives us a function of type `(a -> a)`.

```haskell
twice twice twice :: a -> a
```

This final expression is a function that takes a value of type `a` and returns a value of type `a`.

3. Using **explicit recursion**, implement function `studentsWithB :: [Student] -> [Name]` that returns a list of names of students with `B` grade:

```haskell
studensWithB [Student "Jack" C, Student "Jane" B]
-- ["Jane"]
```

Requirement: you **cannot** use (==) to compare grades for equality.

In [16]:
studentsWithB :: [Student] -> [Name]
studentsWithB [] = []  -- Base case: empty list returns empty result
studentsWithB (Student name B : xs) = name : studentsWithB xs  -- If grade is B, add name
studentsWithB (Student _ _ : xs) = studentsWithB xs  -- If grade is not B, skip and recurse

studentsWithB [Student "Jack" C, Student "Jane" B]

["Jane"]

4. (a) Implement a polymorphic higher-order functions

```haskell
lookupName :: (Name -> Bool) -> [(Name, a)] -> Status a
```

That looks up a value of type `a` paired with a name that satisfies the given predicate. If
more than one name satisfies the predicate, then the first entry is used, while names from
all others are added to the warning list:

```haskell
ages :: [(Name, Int)]
ages = [("John", 21), ("Jack", 23), ("Jane", 22), ("Jenny", 21)]
>>> lookupName (== "Jack") ages
Warn [] 23
>>> lookupName (\name -> length name == 4) ages
Warn ["ignoring entry: Jack","ignoring entry: Jane"] 21
>>> lookupName (\name -> length name == 5) ages
Warn [] 21
>>> lookupName (\name -> length name > 5) ages
Stop
```

In [ ]:
type Name = String

data Status a
  = Stop
  | Warn [String] a
  deriving (Show)

lookupName :: (Name -> Bool) -> [(Name, a)] -> Status a
lookupName _ [] = Stop
lookupName p ((name, value):xs)
  | p name = handleWarnings p name value xs
  | otherwise = lookupName p xs

handleWarnings :: (Name -> Bool) -> Name -> a -> [(Name, a)] -> Status a
handleWarnings _ _ value [] = Warn [] value
handleWarnings p firstName firstValue ((name, value):xs)
  | p name = addWarning name (handleWarnings p firstName firstValue xs)
  | otherwise = handleWarnings p firstName firstValue xs

addWarning :: Name -> Status a -> Status a
addWarning name (Warn warnings value) = Warn (("ignoring entry: " ++ name) : warnings) value
addWarning _ status = status


ages :: [(Name, Int)]
ages = [("John", 21), ("Jack", 23), ("Jane", 22), ("Jenny", 21)]

print (lookupName (== "Jack") ages)
print (lookupName (\name -> length name == 4) ages)
print (lookupName (\name -> length name == 5) ages)
print (lookupName (\name -> length name > 5) ages)

Warn [] 23

Warn ["ignoring entry: Jack","ignoring entry: Jane"] 21

Warn [] 21

Stop

(b) Implement a polymorphic higher-order function
    
```haskell
    produceAll :: (a -> Status a) -> a -> [a]
```
    
that applies a function repeatedly, as long as the result is a `Warn`, collecting all produced values of type `a` and returning them in a list.

```haskell
fizzbuzz :: Integer -> Status Integer
fizzbuzz n
  | fizz && buzz = Warn ["fizzbuzz"] m
  | fizz = Warn ["fizz"] m
  | buzz = Warn ["buzz"] m
  | otherwise = fizzbuzz m
  where
    m = n + 1
    fizz = m `mod` 3 == 0
    buzz = m `mod` 5 == 0

>>> take 10 (produceAll fizzbuzz 0)
[0,3,5,6,9,10,12,15,18,20]
>>> produceAll (\n -> if n < 15 then fizzbuzz n else Stop) 0
[0,3,5,6,9,10,12,15]
````

In [ ]:
data Status a
  = Stop
  | Warn [String] a
  deriving (Show)

produceAll :: (a -> Status a) -> a -> [a]
produceAll f x = case f x of
  Stop         -> [x]
  Warn _ nextX -> x : produceAll f nextX

fizzbuzz :: Integer -> Status Integer
fizzbuzz n
  | fizz && buzz = Warn ["fizzbuzz"] m
  | fizz = Warn ["fizz"] m
  | buzz = Warn ["buzz"] m
  | otherwise = fizzbuzz m
  where
    m = n + 1
    fizz = m `mod` 3 == 0
    buzz = m `mod` 5 == 0

take 10 (produceAll fizzbuzz 0)

produceAll (\n -> if n < 15 then fizzbuzz n else Stop) 0

[0,3,5,6,9,10,12,15,18,20]

[0,3,5,6,9,10,12,15]

(c) Implement a polymorphic higher-order function

```haskell
produceAllWithWarnings :: (a -> Status a) -> a -> ([String], [a])
```
that applies a function repeatedly, as long as the result is a `Warn`, collecting all produced
values of type `a` and all warnings and returning a pair of two collected lists.

```haskell
>>> produceAllWithWarnings (\n -> if n < 15 then fizzbuzz n else Stop) 0
(["fizz","buzz","fizz","fizz","buzz","fizz","fizzbuzz"],[3,5,6,9,10,12,15])
```

Can you also make your implementation sufficiently lazy?

```haskell
>>> take 7 (fst (produceAllWithWarnings fizzbuzz 0))
["fizz","buzz","fizz","fizz","buzz","fizz","fizzbuzz"]
```

In [ ]:
data Status a
  = Stop
  | Warn [String] a
  deriving (Show)

produceAllWithWarnings :: (a -> Status a) -> a -> ([String], [a])
produceAllWithWarnings f x = case f x of
  Stop         -> ([], [x])
  Warn ws nextX -> let (allWarnings, allValues) = produceAllWithWarnings f nextX
                   in (ws ++ allWarnings, x : allValues) 

fizzbuzz :: Integer -> Status Integer
fizzbuzz n
  | fizz && buzz = Warn ["fizzbuzz"] m
  | fizz = Warn ["fizz"] m
  | buzz = Warn ["buzz"] m
  | otherwise = fizzbuzz m
  where
    m = n + 1
    fizz = m `mod` 3 == 0
    buzz = m `mod` 5 == 0

produceAllWithWarnings (\n -> if n < 15 then fizzbuzz n else Stop) 0

take 7 (fst (produceAllWithWarnings fizzbuzz 0))

(["fizz","buzz","fizz","fizz","buzz","fizz","fizzbuzz"],[0,3,5,6,9,10,12,15])

["fizz","buzz","fizz","fizz","buzz","fizz","fizzbuzz"]

(d) Implement a polymorphic higher-order function

```haskell
applyStatus :: Status (a -> b) -> Status a -> Status b
```

that applies a given function to a given argument when both are available, and also combines
warning lists.

```haskell
operators :: [(Name, Int -> Int)]
operators = [("inc", (+1)), ("double", (*2))]

constants :: [(Name, Int)]
constants = [("x", 3), ("y", 4), ("z", 7)]

>>> applyStatus (lookupName (== "double") operators) (lookupName (== "x") constants)
Warn [] 6
>>> applyStatus (lookupName (const True) operators) (lookupName (== "x") constants)
Warn ["ignoring entry: double"] 4
>>> applyStatus (lookupName (const True) operators) (lookupName (const True) constants)
Warn ["ignoring entry: double","ignoring entry: y","ignoring entry: z"] 4
>>> applyStatus (lookupName (== "triple") operators) (lookupName (== "x") constants)
Stop
```

In [ ]:
data Status a
  = Stop
  | Warn [String] a
  deriving (Show)

applyStatus :: Status (a -> b) -> Status a -> Status b
applyStatus Stop _ = Stop
applyStatus _ Stop = Stop
applyStatus (Warn wFunc f) (Warn wVal x) = Warn (wFunc ++ wVal) (f x)

operators :: [(Name, Int -> Int)]
operators = [("inc", (+1)), ("double", (*2))]

constants :: [(Name, Int)]
constants = [("x", 3), ("y", 4), ("z", 7)]

lookupName :: (Name -> Bool) -> [(Name, a)] -> Status a
lookupName _ [] = Stop
lookupName p ((name, value):xs)
  | p name = handleWarnings p name value xs
  | otherwise = lookupName p xs

handleWarnings :: (Name -> Bool) -> Name -> a -> [(Name, a)] -> Status a
handleWarnings _ _ value [] = Warn [] value
handleWarnings p firstName firstValue ((name, value):xs)
  | p name = addWarning name (handleWarnings p firstName firstValue xs)
  | otherwise = handleWarnings p firstName firstValue xs

addWarning :: Name -> Status a -> Status a
addWarning name (Warn warnings value) = Warn (("ignoring entry: " ++ name) : warnings) value
addWarning _ status = status

applyStatus (lookupName (== "double") operators) (lookupName (== "x") constants)
applyStatus (lookupName (const True) operators) (lookupName (== "x") constants)
applyStatus (lookupName (const True) operators) (lookupName (const True) constants)
applyStatus (lookupName (== "triple") operators) (lookupName (== "x") constants)


Warn [] 6

Warn ["ignoring entry: double"] 4

Warn ["ignoring entry: double","ignoring entry: y","ignoring entry: z"] 4

Stop